## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?


> -0.11




In [23]:
import numpy as np
from fastembed import TextEmbedding

# Load the embedding model
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Embed the query
query_1 = ["I just discovered the course. Can I join now?"]
embedding = next(embedder.embed(query_1))  # Returns a generator, so we use `next`

# Convert to numpy array if needed (usually it's already a NumPy array)
embedding = np.array(embedding)

# Check the minimum value
print("Minimum value in embedding:", embedding.min())
print("Shape of embedding:", embedding.shape)

Minimum value in embedding: -0.11726373885183883
Shape of embedding: (512,)


## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can chech that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)

In [7]:
print(f"Norm:\n{np.linalg.norm(embedding)}")  # This will compute the L2 norm of the embedding

print(f"Dot product:\n{embedding.dot(embedding)}")  # This will compute the dot product of the embedding with itself

Norm:
1.0
Dot product:
1.0000000000000002


## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

> 0.9

In [ ]:
query = ["Can I still join the course after the start date?"]
embedding_2 = next(embedder.embed(query))

print(f"Norm of second embedding:\n{np.linalg.norm(embedding_2)}")  # L2 norm of the second embedding
print(f"Dot product of second embedding:\n{embedding_2.dot(embedding_2)}")  # Dot product of the second embedding with itself
print(f"Dot product of first and second embeddings:\n{embedding.dot(embedding_2)}")  # Dot product between the two embeddings
print(f"Cosine similarity:\n{embedding.dot(embedding_2) / (np.linalg.norm(embedding) * np.linalg.norm(embedding_2))}")  # Cosine similarity (same as dot product for normalized vectors)

Norm of second embedding:
1.0
Dot product of second embedding:
1.0
Dot product of first and second embeddings:
0.9008528895674548
Cosine similarity:
0.9008528895674548


## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

> 1

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

In [9]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
# Compute embeddings for the text field in the documents
embeddings = embedder.embed([doc['text'] for doc in documents])

# Compute the cosine similarity between the query embedding and each document embedding
similarities = []
for doc_embedding in embeddings:
    doc_embedding = np.array(doc_embedding)  # Ensure it's a NumPy array
    cosine_similarity = embedding.dot(doc_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(doc_embedding))
    similarities.append(cosine_similarity)

# Print the document with the highest similarity, its index, and the similarity score
max_index = np.argmax(similarities)
print(f"Most similar document index: {max_index}")
print(f"Most similar document text: {documents[max_index]['text']}")
print(f"Cosine similarity score: {similarities[max_index]}")


Most similar document index: 1
Most similar document text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.
Cosine similarity score: 0.8182378156620136


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

> 0

Is it different from Q3? If yes, why?
> Yes. The additional context in question + text helps the embedding model better match intent and meaning.

In [18]:
for doc in documents:
    doc['full_text'] = doc['question'] + ' ' + doc['text']

# Compute embeddings for the full_text field in the documents
embeddings_full_text = embedder.embed([doc['full_text'] for doc in documents])

# Compute the cosine similarity between the query embedding and each document embedding
similarities = []
for doc_embedding in embeddings_full_text:
    doc_embedding = np.array(doc_embedding)  # Ensure it's a NumPy array
    cosine_similarity = embedding.dot(doc_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(doc_embedding))
    similarities.append(cosine_similarity)

# Print the document with the highest similarity, its index, and the similarity score
max_index = np.argmax(similarities)
print(f"Most similar document index: {max_index}")
print(f"Most similar document text: {documents[max_index]['text']}")
print(f"Cosine similarity score: {similarities[max_index]}")

Most similar document index: 0
Most similar document text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
Cosine similarity score: 0.8514543194432261


## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

> 384

One of these models is `BAAI/bge-small-en`. Let's use it.

In [19]:
# Check the shape of the embeddings for model BAAI/bge-small-en
# Load the embedding model
embedder_baai = TextEmbedding(model_name="BAAI/bge-small-en")

# Embed the query using the BAAI model
embedding_baai = next(embedder_baai.embed(query))  # Returns a generator, so we use `next`
# Convert to numpy array if needed (usually it's already a NumPy array)
embedding_baai = np.array(embedding_baai)
# Check the shape of the embedding
print("Shape of embedding from BAAI model:", embedding_baai.shape)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

Shape of embedding from BAAI model: (384,)


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:


In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        doc['text'] = doc['question'] + ' ' + doc['text']
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):


> 0.87



In [41]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np

# Connect to Qdrant (local)
client = QdrantClient("http://localhost:6333")

# Model name and FastEmbed instance
model_handle = "BAAI/bge-small-en"
embedder_baai = TextEmbedding(model_name=model_handle)

# Collection name
collection_name = "zoomcamp-homework-2"

# Recreate the collection with correct vector size and distance
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # BAAI/bge-small-en dimensionality
        distance=models.Distance.COSINE
    )
)

# Prepare and insert points with full_text = question + text
points = []
for idx, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    vector = next(embedder_baai.embed([full_text]))  # Local embedding
    point = models.PointStruct(
        id=idx,
        vector=vector,
        payload={
            "question": doc["question"],
            "text": doc["text"],
            "course": doc["course"]
        }
    )
    points.append(point)

# Upsert points into Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)


/var/folders/55/x1yqz6851xj6j2f0smdhs4jh0000gn/T/ipykernel_58798/1691277066.py:16: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [42]:
query_vector = next(embedder_baai.embed(["I just discovered the course. Can I join now?"]))

search_result = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
    with_payload=True
)

# Print the top result and score
print("Top document:", search_result[0].payload["text"])
print("Cosine similarity score:", search_result[0].score)


Top document: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
Cosine similarity score: 0.87031734


/var/folders/55/x1yqz6851xj6j2f0smdhs4jh0000gn/T/ipykernel_58798/541462779.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(
